In [16]:
from IPython.display import Code

Code(filename='cluster.tf', language='terraform')

terraform {
  required_providers {
    linode = {
      source = "linode/linode"
      # version = "..."
    }
  }
}

variable "linode_token" {
  type      = string
  sensitive = true
}

# Configure the Linode Provider
provider "linode" {
  // Linode Personal Access Token from environment variable
  token = var.linode_token

}

resource "linode_lke_cluster" "pgo-demo" {
  label       = "pgo-demo"
  k8s_version = "1.29"
  region      = "fr-par"

  pool {
    type  = "g6-standard-2"
    count = 2
  }
}

# Output the kubeconfig in a file after base64 decoding
resource "local_file" "kubeconfig" {
  content  = base64decode(linode_lke_cluster.pgo-demo.kubeconfig)
  filename = "kubeconfig.yaml"
}

In [17]:
!terraform apply -auto-approve


Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # linode_lke_cluster.pgo-demo will be created
  + resource "linode_lke_cluster" "pgo-demo" {
      + api_endpoints = (known after apply)
      + dashboard_url = (known after apply)
      + id            = (known after apply)
      + k8s_version   = "1.29"
      + kubeconfig    = (sensitive value)
      + label         = "pgo-demo"
      + region        = "fr-par"
      + status        = (known after apply)
      + tags          = (known after apply)

      + control_plane (known after apply)

      + pool {
          + count           = 2
          + disk_encryption = (known after apply)
          + id              = (known after apply)
          + nodes           = (known after apply)
          + type            = "g6-standard-2"
        }
    }

  # local_file.kubeconfig will be created

error: lost connection to pod


linode_lke_cluster.pgo-demo: Still creating... [30s elapsed]
linode_lke_cluster.pgo-demo: Still creating... [40s elapsed]
linode_lke_cluster.pgo-demo: Still creating... [50s elapsed]
linode_lke_cluster.pgo-demo: Still creating... 0s elapsed]
linode_lke_cluster.pgo-demo: Creation complete after 1m4s [id=245565]
local_file.kubeconfig: Creating...
local_file.kubeconfig: Creation complete after 0s [id=e8b4391633b68f0d3be6acafc4ddbbda02943d17]

Apply complete! Resources: 2 added, 0 changed, 0 destroyed.


0

In [ ]:
from IPython.display import Code

Code(filename='kubeconfig.yaml', language='yaml')

In [21]:
!kubectl config view

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: DATA+OMITTED
    server: https://441c51eb-da7a-4af6-970b-5a79cd515f80.fr-par-2.linodelke.net:443
  name: lke245474
contexts:
- context:
    cluster: lke245474
    namespace: default
    user: lke245474-admin
  name: lke245474-ctx
current-context: lke245474-ctx
kind: Config
preferences: {}
users:
- name: lke245474-admin
  user:
    token: REDACTED


In [23]:
!kubectl apply -f https://github.com/parca-dev/parca/releases/download/v0.22.0/kubernetes-manifest.yaml

configmap/parca created
deployment.apps/parca created
namespace/parca unchanged
service/parca unchanged
serviceaccount/parca unchanged


In [24]:
!kubectl apply -f https://github.com/parca-dev/parca-agent/releases/download/v0.33.1/kubernetes-manifest.yaml

namespace/parca unchanged
clusterrole.rbac.authorization.k8s.io/parca-agent created
clusterrolebinding.rbac.authorization.k8s.io/parca-agent created
configmap/parca-agent created
daemonset.apps/parca-agent created
serviceaccount/parca-agent created


In [25]:
!kubectl get pods -n parca -o wide -w

NAME                     READY   STATUS    RESTARTS   AGE   IP         NODE                            NOMINATED NODE   READINESS GATES
parca-77f54b74c7-wf26x   0/1     Running   0          17s   10.2.0.6   lke245474-389357-4cdbdaac0000   <none>           <none>
parca-agent-8t2gk        1/1     Running   0          8s    10.2.0.7   lke245474-389357-4cdbdaac0000   <none>           <none>
parca-77f54b74c7-wf26x   1/1     Running   0          20s   10.2.0.6   lke245474-389357-4cdbdaac0000   <none>           <none>
^C


In [2]:
import os
get_ipython().system = os.system
!kubectl -n parca port-forward service/parca 7070 & echo $! > /tmp/parca-forward.pid

0

Forwarding from 127.0.0.1:7070 -> 7070
Forwarding from [::1]:7070 -> 7070
Handling connection for 7070
Handling connection for 7070
Handling connection for 7070
Handling connection for 7070
Handling connection for 7070
Handling connection for 7070
Handling connection for 7070


In [ ]:
!kill $(cat /tmp/parca-forward.pid)

In [27]:
from IPython.display import display, HTML

display(HTML('<a href="http://localhost:7070" target="_blank">Open Parca UI</a>'))

Handling connection for 7070
Handling connection for 7070
Handling connection for 7070


In [18]:
!cd demo; go build -o demo .

0

In [ ]:
Code(filename='demo/Dockerfile', language='Docker')

In [50]:
!docker build --platform linux/amd64 -t ghcr.io/guilhem/pgo-demo:nopgo -f demo/Dockerfile --push demo/

#0 building with "orbstack" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 359B done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/alpine:3.14
#2 DONE 1.1s

#3 [internal] load metadata for docker.io/library/golang:1.23
#3 DONE 1.2s

#4 [internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [builder 1/6] FROM docker.io/library/golang:1.23@sha256:a7f2fc9834049c1f5df787690026a53738e55fc097cd8a4a93faa3e06c67ee32
#5 DONE 0.0s

#6 [stage-1 1/2] FROM docker.io/library/alpine:3.14@sha256:0f2d5c38dd7a4f4f733e688e3a6733cb5ab1ac6e3cb4603a5dd564e5bfb80eed
#6 DONE 0.0s

#7 [internal] load build context
#7 transferring context: 9.24MB 0.1s done
#7 DONE 0.1s

#8 [builder 3/6] COPY go.mod go.sum ./
#8 CACHED

#9 [builder 2/6] WORKDIR /go/src/app
#9 CACHED

#10 [builder 4/6] RUN go mod download
#10 CACHED

#11 [builder 5/6] COPY . .
#11 DONE 0.2s

#12 [builder 6/6] RUN go build -o /go/bin/app -ldfl

0

In [53]:
!kubectl run pgo-demo --image=ghcr.io/guilhem/pgo-demo:nopgo --port=8080

pod/pgo-demo created


0

In [10]:
import os
get_ipython().system = os.system
!kubectl port-forward pods/pgo-demo 8080 & echo $! > port-forward.pid

0

Forwarding from 127.0.0.1:8080 -> 8080
Forwarding from [::1]:8080 -> 8080


In [ ]:
!kill $(cat port-forward.pid)

In [55]:
!curl -o test.md -L "https://raw.githubusercontent.com/golang/go/c16c2c49e2fa98ae551fc6335215fadd62d33542/README.md"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1455  100  1455    0     0   4504      0 --:--:-- --:--:-- --:--:--  4518


0

In [56]:
!curl --data-binary @test.md http://localhost:8080/render

Handling connection for 8080
<h1>The Go Programming Language</h1>
<p>Go is an open source programming language that makes it easy to build simple,
reliable, and efficient software.</p>
<p><img src="https://golang.org/doc/gopher/fiveyears.jpg" alt="Gopher image" />
<em>Gopher image by <a href="https://reneefrench.blogspot.com/">Renee French</a>, licensed under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons 4.0 Attributions license</a>.</em></p>
<p>Our canonical Git repository is located at <a href="https://go.googlesource.com/go">https://go.googlesource.com/go</a>.
There is a mirror of the repository at <a href="https://github.com/golang/go">https://github.com/golang/go</a>.</p>
<p>Unless otherwise noted, the Go source files are distributed under the
BSD-style license found in the LICENSE file.</p>
<h3>Download and Install</h3>
<h4>Binary Distributions</h4>
<p>Official binary distributions are available at <a href="https://go.dev/dl/">https://go.dev/dl/</a>.</p>

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3307  100  1852  100  1455  74080  58200 --:--:-- --:--:-- --:--:--  129k


0

In [ ]:
!go run github.com/prattmic/markdown-pgo/load@latest

Handling connection for 8080


In [1]:
from IPython.display import display, HTML

url = "http://localhost:7070/?query_browser_mode=advanced&expression_a=parca_agent%3Asamples%3Acount%3Acpu%3Ananoseconds%3Adelta%7Bnamespace%3D%22default%22%2C%20run%3D%22pgo-demo%22%7D&from_a=1729002421615&to_a=1729003321615&time_selection_a=relative:minute|15&sum_by_a=__none__&merge_from_a=1729002421615&merge_to_a=1729003321615&selection_a=parca_agent%3Asamples%3Acount%3Acpu%3Ananoseconds%3Adelta%7Bnamespace%3D%22default%22%2C%20run%3D%22pgo-demo%22%7D"
display(HTML(f'<a href="{url}" target="_blank">Open Parca Query</a>'))

In [3]:
%%bash
cd demo/
mv ~/Downloads/profile.pb.gz ./profile.pb.gz
gunzip -f profile.pb.gz

In [4]:
!cd demo; go build -o demo-pgo -pgo ./profile.pb .

0

In [ ]:
from IPython.display import Code

Code(filename='demo/Dockerfile.pgo', language='Docker')

In [5]:
!docker build --platform linux/amd64 -t ghcr.io/guilhem/pgo-demo:pgo -f demo/Dockerfile.pgo --push demo/

#0 building with "orbstack" instance using docker driver

#1 [internal] load build definition from Dockerfile.pgo
#1 transferring dockerfile: 379B done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/golang:1.23
#2 ...

#3 [internal] load metadata for docker.io/library/alpine:3.14
#3 DONE 1.7s

#2 [internal] load metadata for docker.io/library/golang:1.23
#2 DONE 1.8s

#4 [internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [builder 1/6] FROM docker.io/library/golang:1.23@sha256:a7f2fc9834049c1f5df787690026a53738e55fc097cd8a4a93faa3e06c67ee32
#5 DONE 0.0s

#6 [stage-1 1/2] FROM docker.io/library/alpine:3.14@sha256:0f2d5c38dd7a4f4f733e688e3a6733cb5ab1ac6e3cb4603a5dd564e5bfb80eed
#6 DONE 0.0s

#7 [internal] load build context
#7 transferring context: 9.20MB 0.0s done
#7 DONE 0.1s

#8 [builder 2/6] WORKDIR /go/src/app
#8 CACHED

#9 [builder 3/6] COPY go.mod go.sum ./
#9 CACHED

#10 [builder 4/6] RUN go mod download
#10 CACHED

#11 [builder 5/6] C

0

In [6]:
!kubectl run pgo-demo-pgo --image=ghcr.io/guilhem/pgo-demo:pgo --port=8080

pod/pgo-demo-pgo created


0

In [7]:
import os
get_ipython().system = os.system
!kubectl port-forward pods/pgo-demo-pgo 8081:8080 & echo $! > port-forward-pgo.pid

0

Forwarding from 127.0.0.1:8081 -> 8080
Forwarding from [::1]:8081 -> 8080


In [1]:
!kill $(cat port-forward-pgo.pid)

In [8]:
!go get github.com/prattmic/markdown-pgo/load@latest

go: added github.com/prattmic/markdown-pgo v0.0.0-20230831075821-97e1a165aa5f


0

In [11]:
! go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr http://localhost:8080 > nopgo.txt
from IPython.display import Code
Code(filename='nopgo.txt')

Handling connection for 8080


goos: darwin
goarch: arm64
pkg: github.com/prattmic/markdown-pgo/load
cpu: Apple M3
BenchmarkLoad-8   	     142	   8715645 ns/op
BenchmarkLoad-8   	     141	   8623604 ns/op
BenchmarkLoad-8   	     145	   8559082 ns/op
BenchmarkLoad-8   	     140	   8428731 ns/op
BenchmarkLoad-8   	     139	   8442536 ns/op
BenchmarkLoad-8   	     136	   8478911 ns/op
BenchmarkLoad-8   	     142	   8576510 ns/op
BenchmarkLoad-8   	     138	   8688456 ns/op
BenchmarkLoad-8   	     141	   8431836 ns/op
BenchmarkLoad-8   	     139	   8575292 ns/op
BenchmarkLoad-8   	     141	   8403759 ns/op
BenchmarkLoad-8   	     141	   8485898 ns/op
BenchmarkLoad-8   	     141	   8666989 ns/op
BenchmarkLoad-8   	     138	   8714442 ns/op
BenchmarkLoad-8   	     139	   8575520 ns/op
BenchmarkLoad-8   	     139	   8693032 ns/op
BenchmarkLoad-8   	     139	   8448964 ns/op
BenchmarkLoad-8   	     135	   8807743 ns/op
BenchmarkLoad-8   	     139	   8452234 ns/op
BenchmarkLoad-8   	     141	   8729340 ns/op
BenchmarkLoad-8   	     144	   8553559 ns/op
BenchmarkLoad-8   	     136	   8482243 ns/op
BenchmarkLoad-8   	     138	   8543142 ns/op
BenchmarkLoad-8   	     122	   8646490 ns/op
BenchmarkLoad-8   	     138	   8683020 ns/op
BenchmarkLoad-8   	     139	   8537712 ns/op
BenchmarkLoad-8   	     142	   8626035 ns/op
BenchmarkLoad-8   	     142	   8625999 ns/op
BenchmarkLoad-8   	     138	   8696025 ns/op
BenchmarkLoad-8   	     136	   8609539 ns/op
BenchmarkLoad-8   	     136	   8480888 ns/op
BenchmarkLoad-8   	     134	   8573819 ns/op
BenchmarkLoad-8   	     136	   8453977 ns/op
BenchmarkLoad-8   	     140	   8472540 ns/op
BenchmarkLoad-8   	     134	   8489050 ns/op
BenchmarkLoad-8   	     141	   8550195 ns/op
BenchmarkLoad-8   	     141	   8669567 ns/op
BenchmarkLoad-8   	     135	   8547195 ns/op
BenchmarkLoad-8   	     142	   8577397 ns/op
BenchmarkLoad-8   	     139	   8935416 ns/op
PASS
ok  	github.com/prattmic/markdown-pgo/load	82.072s

In [12]:
!go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr "http://localhost:8081" > withpgo.txt

from IPython.display import Code
Code(filename='withpgo.txt')

Handling connection for 8081


goos: darwin
goarch: arm64
pkg: github.com/prattmic/markdown-pgo/load
cpu: Apple M3
BenchmarkLoad-8   	     134	   8291251 ns/op
BenchmarkLoad-8   	     146	   8499089 ns/op
BenchmarkLoad-8   	     139	   8771565 ns/op
BenchmarkLoad-8   	     141	   8652417 ns/op
BenchmarkLoad-8   	     138	   8787800 ns/op
BenchmarkLoad-8   	     142	   8415208 ns/op
BenchmarkLoad-8   	     139	   8653329 ns/op
BenchmarkLoad-8   	     133	  10330742 ns/op
BenchmarkLoad-8   	     133	   8581844 ns/op
BenchmarkLoad-8   	     140	   8566208 ns/op
BenchmarkLoad-8   	     140	   8711599 ns/op
BenchmarkLoad-8   	     140	   8657724 ns/op
BenchmarkLoad-8   	     139	   9030309 ns/op
BenchmarkLoad-8   	     132	   9005391 ns/op
BenchmarkLoad-8   	     135	   8618515 ns/op
BenchmarkLoad-8   	     139	   8555788 ns/op
BenchmarkLoad-8   	     138	   9119862 ns/op
BenchmarkLoad-8   	     133	   8597729 ns/op
BenchmarkLoad-8   	     142	   8504548 ns/op
BenchmarkLoad-8   	     136	   8453962 ns/op
BenchmarkLoad-8   	     139	   8596076 ns/op
BenchmarkLoad-8   	     139	   8373917 ns/op
BenchmarkLoad-8   	     141	   8779497 ns/op
BenchmarkLoad-8   	     141	   8560864 ns/op
BenchmarkLoad-8   	     140	   8425115 ns/op
BenchmarkLoad-8   	     144	   8499075 ns/op
BenchmarkLoad-8   	     139	   8382801 ns/op
BenchmarkLoad-8   	     140	   8589376 ns/op
BenchmarkLoad-8   	     138	   8500852 ns/op
BenchmarkLoad-8   	     141	   8621146 ns/op
BenchmarkLoad-8   	     140	   8586607 ns/op
BenchmarkLoad-8   	     138	   8412371 ns/op
BenchmarkLoad-8   	     139	   8339606 ns/op
BenchmarkLoad-8   	     142	   8603090 ns/op
BenchmarkLoad-8   	     100	  10142140 ns/op
BenchmarkLoad-8   	     124	   8782318 ns/op
BenchmarkLoad-8   	     141	   8407178 ns/op
BenchmarkLoad-8   	     139	   8863707 ns/op
BenchmarkLoad-8   	     142	   8421548 ns/op
BenchmarkLoad-8   	     140	   8789227 ns/op
PASS
ok  	github.com/prattmic/markdown-pgo/load	81.752s

In [13]:
!go install golang.org/x/perf/cmd/benchstat@latest

0

In [14]:
%%bash
benchstat nopgo.txt withpgo.txt

goos: darwin
goarch: arm64
pkg: github.com/prattmic/markdown-pgo/load
cpu: Apple M3
       │  nopgo.txt  │          withpgo.txt          │
       │   sec/op    │   sec/op     vs base          │
Load-8   8.575m ± 1%   8.593m ± 1%  ~ (p=0.543 n=40)


In [15]:
!terraform destroy -auto-approve

linode_lke_cluster.pgo-demo: Refreshing state... [id=245474]
local_file.kubeconfig: Refreshing state... [id=54a7a892a081868681d9879efa309a526e67b7c4]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # linode_lke_cluster.pgo-demo will be destroyed
  - resource "linode_lke_cluster" "pgo-demo" {
      - api_endpoints = [
          - "https://441c51eb-da7a-4af6-970b-5a79cd515f80.fr-par-2.linodelke.net:443",
          - "https://441c51eb-da7a-4af6-970b-5a79cd515f80.fr-par-2.linodelke.net:6443",
          - "https://172.232.48.54:443",
          - "https://172.232.48.54:6443",
        ] -> null
      - dashboard_url = "https://144c6d58-aad4-4fb1-b34d-1a347024f3bc.dashboard.fr-par-2.linodelke.net" -> null
      - id            = "245474" -> null
      - k8s_version   = "1.29" -> null
      - kubeconfig    = (sensitive value) -> null
      - l

0

error: lost connection to pod
error: lost connection to pod


In [19]:
# run ./demo/demo
! go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr http://localhost:8080 > nopgo-local.txt

0

In [20]:
# run ./demo/demo-pgo
! go test github.com/prattmic/markdown-pgo/load -bench=. -count=40 -source $(pwd)/test.md -addr http://localhost:8080 > withpgo-local.txt

0

In [21]:
%%bash
benchstat nopgo-local.txt withpgo-local.txt

goos: darwin
goarch: arm64
pkg: github.com/prattmic/markdown-pgo/load
cpu: Apple M3
       │ nopgo-local.txt │         withpgo-local.txt          │
       │     sec/op      │   sec/op     vs base               │
Load-8       114.2µ ± 0%   114.8µ ± 1%  +0.47% (p=0.000 n=40)
